In [2]:
import os
import glob
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import sys
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
# Recall y precision
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]
Spark version: 3.1.2


In [6]:
path='./DATA'
file = os.path.join(path, "users_bg.csv")

In [7]:
df=pd.read_csv(file, usecols=['u_id','bgg_id','Game','category','Your Rating'], low_memory=False)
df.head()

,u_id,Game,Your Rating,bgg_id,category
0,1,1830: Railways & Robber Barons,-1,421,"1021,1034,1011"
1,1,18AL,7,2612,"1021,1120,1034,1011"
2,1,2 de Mayo,6,36522,"1051,1019"
3,1,23,7,103651,1002
4,1,6 nimmt!,7,432,"1002,1098"


In [8]:
#Transform explicit values to implicit values
df['Rating'] = df['Your Rating'].apply(lambda x: 1 if x>0 else 0)